<a href="https://colab.research.google.com/github/kyin99/NSAC-CyberScape/blob/master/flood_of_ideas_(final).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models

In [3]:
path = "/content/drive/My Drive/Colab Notebooks/trainfinal.csv"
train_dataset=pd.read_csv(path)
p = "/content/drive/My Drive/Colab Notebooks/testfinal.csv"
test_dataset=pd.read_csv(p)

In [5]:
train_dataset.head()

,RUNUP_HT,DIST_FROM_SOURCE,CAUSE_CODE,TS_INTENSITY,DAMAGE_AMOUNT_ORDER
0,NaN,93.0,0.0,5.0,4.0
1,10.0,137.0,1.0,5.0,4.0
2,NaN,23.0,1.0,5.0,4.0
3,85.4,40.0,1.0,6.0,4.0
4,12.0,131.0,2.0,4.0,4.0


In [6]:
train_features = train_dataset.copy()
train_labels = train_features.pop('DAMAGE_AMOUNT_ORDER')
test_features = test_dataset.copy()
test_labels = test_features.pop('DAMAGE_AMOUNT_ORDER')